# Track Cluster

In [1]:
import pandas as pd
import numpy as np
from utils import utils
from utils import myplot
df = pd.read_csv(r'wifi_track_data\dacang\track_data\dacang_track_data_3_final_1220.csv')
df.t = pd.to_datetime(df.t)
df_wifipos = pd.read_csv(r'wifi_track_data\dacang\pos_data\wifi_pos_new_3_1220.csv')
from tqdm import tqdm
from importlib import reload

import warnings
warnings.filterwarnings('ignore')

## 初步分析
每个date_mac的:
- 数据量
- 探针数量
- 一天内持续时间
- 一天内发生活动的平均时刻
- 一天内的总移动距离

In [2]:
mac_list = df['m'].unique()
ori_mac_list = df.oriMac.unique()
ori_Mac = []
data_count = []
Dur = []
wifi_count = []
active_moment = []
middle_moment = []
moving_dis = []
for m in mac_list:
    ori_Mac.append(m.split('-')[1])
    df_now = utils.GetDfNow(df, m)
    data_count.append(len(df_now))
    t1 = df_now.iloc[0].t
    t2 = df_now.iloc[len(df_now)-1].t
    t1 = t1.hour + t1.minute/60 + t1.second/3600
    t2 = t2.hour + t2.minute/60 + t2.second/3600
    Dur.append(t2-t1)
    middle_moment.append((t2+t1)/2)
    wifi_count.append(len(df_now['a'].unique()))
    h = 0
    row_last = df_now.iloc[0]
    dis = 0
    for i,row in df_now.iterrows():
        h += row.t.hour + row.t.minute/60 + row.t.second/3600
        if row.a != row_last.a:
            dis += utils.GetWifiTrackDistance(row.a,row_last.a,df_wifipos)
        row_last = row
    active_moment.append(h/len(df_now))
    moving_dis.append(dis)

df_sta = pd.DataFrame({'mac':mac_list,
                       'ori_mac':ori_Mac,
                       'data_count':data_count,
                       'Dur':Dur,
                       'wifi_count':wifi_count,
                       'active_moment':active_moment,
                       'moving_dis':moving_dis,
                       'middle_moment':middle_moment})
df_sta

,mac,ori_mac,data_count,Dur,wifi_count,active_moment,moving_dis,middle_moment
0,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",5,7.137500,3,21.925500,255.84,20.428194
1,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",9,7.140278,5,22.534414,437.00,20.429583
2,"7.5-20,178,229,137,47,142","20,178,229,137,47,142",12,0.133056,2,23.961944,697.56,23.928750
3,"7.6-48,74,38,155,214,98","48,74,38,155,214,98",44,23.999167,14,4.545069,1053.48,12.000139
4,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",11,23.996389,4,10.667727,161.62,11.998750
...,...,...,...,...,...,...,...,...
1339,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",14,16.207500,4,15.388036,2014.83,15.774306
1340,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",32,13.351667,6,16.939488,2149.56,17.204444
1341,"8.7-56,251,20,50,26,70","56,251,20,50,26,70",6,8.996944,3,18.492269,363.15,15.832361
1342,"8.7-144,173,247,200,14,93","144,173,247,200,14,93",10,8.199167,4,18.670944,559.52,15.882361


In [3]:
myplot.Boxes([(df_sta.data_count,"数据量"),
              (df_sta.Dur,"持续时间"),
              (df_sta.wifi_count,"wifi数量"),
              (df_sta.active_moment,"活跃时刻"),
              (df_sta.moving_dis,"移动距离"),
              (df_sta.middle_moment,"中间时刻")])

## 清除离群值
随机孤立森林

In [4]:
#归一化
df_outlier = df_sta[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']]
df_outlier = utils.Normalize_df(df_outlier,['data_count','Dur','active_moment','moving_dis','wifi_count'],mul_index=10)
df_outlier

,data_count,Dur,active_moment,moving_dis,wifi_count,middle_moment
0,0.140845,2.971261,9.132320,0.281856,0.740741,20.428194
1,0.422535,2.972419,9.391764,0.481438,1.481481,20.429583
2,0.633803,0.051526,10.000000,0.768493,0.370370,23.928750
3,2.887324,9.999884,1.726939,1.160606,4.814815,12.000139
4,0.563380,9.998726,4.335655,0.178055,1.111111,11.998750
...,...,...,...,...,...,...
1339,0.774648,6.752003,6.346865,2.219714,1.111111,15.774306
1340,2.042254,5.561577,7.007901,2.368144,1.851852,17.204444
1341,0.211268,3.746353,7.669503,0.400078,0.740741,15.832361
1342,0.492958,3.413807,7.745633,0.616416,1.111111,15.882361


In [5]:
from sklearn.ensemble import IsolationForest
rng = np.random.RandomState(42)
model_IF = IsolationForest(max_samples=500,random_state=rng,contamination='auto')
model_IF.fit(df_outlier[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']])
df_outlier['label'] = model_IF.predict(df_outlier[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']])
print(df_outlier.label.value_counts())

myplot.Scatter_Matrix(df_outlier,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'label')

label
 1    1151
-1     193
Name: count, dtype: int64


In [6]:
df_cluster = df_sta[['mac','ori_mac','data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']][df_outlier.label == 1].reset_index(drop=True)
df_cluster = utils.Normalize_df(df_cluster,['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment'],mul_index=10)

df_cluster

,mac,ori_mac,data_count,Dur,active_moment,moving_dis,wifi_count,middle_moment
0,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",0.714286,2.968057,9.885586,1.104268,2.500000,9.167477
1,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",1.000000,9.998841,3.638626,0.388653,1.666667,4.518370
2,"7.6-48,74,38,133,29,213","48,74,38,133,29,213",1.571429,9.998841,3.031398,1.481773,3.333333,4.518370
3,"7.6-20,178,229,137,47,142","20,178,229,137,47,142",0.285714,9.997799,3.906912,0.260748,0.833333,4.517834
4,"7.6-4,207,140,11,128,186","4,207,140,11,128,186",0.857143,9.995018,2.679807,0.541194,0.833333,4.519366
...,...,...,...,...,...,...,...,...
1146,"8.7-76,2,32,111,66,59","76,2,32,111,66,59",0.285714,4.140356,5.210724,0.971815,1.666667,4.851685
1147,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",1.428571,6.750049,6.123530,5.204487,1.666667,6.600366
1148,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",4.000000,5.558864,6.940259,5.554603,3.333333,7.389003
1149,"8.7-56,251,20,50,26,70","56,251,20,50,26,70",0.285714,3.742483,7.757686,0.912358,0.833333,6.632380


In [7]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    'wifi_count'])

## 简单的居民、游客、路过人群分类
- 当一天内待在区域超过10小时的天数超过10天时，被视为居民
- 当一天内待在区域小于0.5小时的天数超过5天时，被视为路过
- 其余视为游客

In [8]:
def DivideId(df_now):
    days = df_now.calender.unique()
    count = 0
    isPassing = 0
    for i in range(len(days)):
        df_one = df_now[df_now.calender == days[i]]
        time = df_one.iloc[len(df_one)-1].t - df_one.iloc[0].t
        h = time.components.hours
        if h>5:
            count += 1
        elif h<0.5:
            isPassing += 1
    if isPassing > 5:
        return 'pass'
    elif count < 4:
        return 'tourist'
    else:
        return 'resident'
    

tourist_list = []
resident_list = []
pass_list = []

for mac in tqdm(ori_mac_list,desc="Dividing:"):
    df_now = df[df.oriMac == mac]
    df_now.sort_values(by='t')
    result = DivideId(df_now)
    if result == 'pass':
        pass_list.append(mac)
    elif result == 'tourist':
        tourist_list.append(mac)
    else:
        resident_list.append(mac)

# add id to df_sta
def GetId(m):
    if m in tourist_list:
        return 1
    elif m in resident_list:
        return 0
    else:
        return -1
df_sta['id'] = df_sta.ori_mac.apply(GetId)
df_cluster['id'] = df_sta.ori_mac.apply(GetId)

print('pass count:',len(pass_list))
print('tourists count:',len(tourist_list))
print('residents count:',len(resident_list))


Dividing:: 100%|██████████| 316/316 [00:00<00:00, 404.09it/s]

pass count: 0
tourists count: 236
residents count: 80


In [9]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'id')

## K-means++聚类

In [10]:
#获得居民最佳聚类数
df_resident = df_cluster[df_cluster.id == 0]
df_score_resi = utils.GetKmeansClusterNumDf(df_resident,['Dur','data_count','active_moment','moving_dis','middle_moment'])
myplot.Double_Axes_Line(df_score_resi,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [11]:
#获得游客最佳聚类数
df_tourist = df_cluster[df_cluster.id == 1]
df_score_tour = utils.GetKmeansClusterNumDf(df_tourist,['Dur','data_count','active_moment','moving_dis','middle_moment'])
myplot.Double_Axes_Line(df_score_tour,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [12]:
df_resident['cluster'] = utils.Kmeans(df_resident,['Dur','data_count','active_moment','moving_dis','middle_moment'],4)
myplot.Scatter_Matrix(df_resident,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'cluster',
                        color_scale='Jet')

In [15]:
df_tourist['cluster'] = utils.Kmeans(df_tourist,['Dur','data_count','active_moment','moving_dis','middle_moment'],4)
myplot.Scatter_Matrix(df_tourist,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'cluster',
                        color_scale='Jet')

In [16]:
df_resident.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_resident.csv",index=False)
df_tourist.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_tourist.csv",index=False)